In [1]:
import pandas as pd
import numpy as np

In [2]:
%matplotlib inline
import  matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set(rc = {'figure.figsize' : (9, 6)})

In [457]:
events_data = pd.read_csv( 'D:\\КУРСЫ DATA SCIENCE\\МОИ ЗАДАЧКИ\\event_data_train.csv')
submissions_data = pd.read_csv( 'D:\\КУРСЫ DATA SCIENCE\\МОИ ЗАДАЧКИ\\submissions_data_train.csv')

In [458]:
events_data.head( )

,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632
1,32815,1434340848,passed,17632
2,32815,1434340848,discovered,17632
3,32811,1434340895,discovered,17632
4,32811,1434340895,viewed,17632


In [459]:
submissions_data.head( )

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


Добавим колонку с датой, переведя timestamp из секунд в нормальную дату

In [460]:
events_data['date'] = pd.to_datetime(events_data.timestamp, unit = 's') 
events_data.head ( )

,step_id,timestamp,action,user_id,date
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48
1,32815,1434340848,passed,17632,2015-06-15 04:00:48
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35


In [461]:
events_data['day'] = events_data.date.dt.date
events_data.head (10)

,step_id,timestamp,action,user_id,date,day
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15
5,32811,1434340895,passed,17632,2015-06-15 04:01:35,2015-06-15
6,32929,1434340921,discovered,17632,2015-06-15 04:02:01,2015-06-15
7,32929,1434340921,viewed,17632,2015-06-15 04:02:01,2015-06-15
8,32929,1434340939,started_attempt,17632,2015-06-15 04:02:19,2015-06-15
9,32929,1434340944,passed,17632,2015-06-15 04:02:24,2015-06-15


In [462]:
now = 1526772811
drop_out_threshold = 2592000

In [463]:
users_data = events_data.groupby('user_id', as_index = False)\
    .agg({'timestamp' : 'max'}).rename(columns = {'timestamp' : 'last_timestamp'})

In [464]:
users_data['is_gone_user'] = (now - users_data.last_timestamp) > drop_out_threshold 
# Сколько времени прошло с последнего посещения пользователя по сравнению с нашим текущим таймстепом( проверим, разность больше ,чем наше 
# пороговое значение в 30 дней)`

In [465]:
users_scores = submissions_data.pivot_table(index = 'user_id', 
                                    columns = 'submission_status',
                                    values = 'step_id',
                                    aggfunc = 'count', 
                                    fill_value = 0).reset_index( )
users_scores.head ( )
# Кол-во корректных и некорректных( не верных) сабмитов пользователей

submission_status,user_id,correct,wrong
0,2,2,0
1,3,29,23
2,5,2,2
3,8,9,21
4,14,0,1


In [466]:
users_data = users_data.merge(users_scores,on = 'user_id',how = 'outer')
# сращивание 2х датафреймов при учете не одинаковых данных, строк (в данном случае юзер айди =1)( on = user_id - по какому критерию соединяются датафреймы)
# inner - теряет много строк, т.к. не учитывает не совпадающие строки

In [467]:
users_data = users_data.fillna(0)

In [468]:
user_min_time=events_data.groupby('user_id',  as_index = False) \
    .agg({'timestamp': 'min' }) \
    .rename({'timestamp': 'min_timestamp'}, axis = 1)

In [469]:
users_data = users_data.merge(user_min_time, how ='outer')

In [470]:
users_data = users_data.fillna(0)

In [471]:
users_events_data = events_data.pivot_table(index = 'user_id', 
                                    columns = 'action',
                                    values = 'step_id',
                                    aggfunc = 'count', 
                                    fill_value = 0).reset_index( )

In [472]:
users_data = users_data.merge(users_events_data, how = 'outer')

In [473]:
users_days = events_data.groupby('user_id').day.nunique( ).to_frame( ).reset_index( )

In [474]:
users_data = users_data.merge(users_days, how = 'outer' )

In [475]:
users_data.drop('wrong', inplace=True, axis=1) # удаление столбца из ДФ

In [476]:
users_data.rename (columns = {'correct_x' : 'correct', 'wrong_x': 'wrong'}). head ( )

,user_id,last_timestamp,is_gone_user,correct,min_timestamp,discovered,passed,started_attempt,viewed,day
0,1,1472827464,True,0.0,1472827464,1,0,0,1,1
1,2,1519226966,True,2.0,1514383364,9,9,2,10,2
2,3,1444581588,True,29.0,1434358476,91,87,30,192,7
3,5,1499859939,True,2.0,1466156809,11,11,4,12,2
4,7,1521634660,True,0.0,1521634660,1,1,0,1,1


In [477]:
users_data['passed_corse'] = users_data.passed > 170

In [478]:
user_min_time.head( )

,user_id,min_timestamp
0,1,1472827464
1,2,1514383364
2,3,1434358476
3,5,1466156809
4,7,1521634660


In [479]:
events_data ['user_time'] = events_data.user_id.map(str) + ' '+ events_data.timestamp.map(str) # операция конкантенации 2х колонок таблицы

In [480]:
events_data.head( )

,step_id,timestamp,action,user_id,date,day,user_time
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,17632 1434340848
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,17632 1434340848
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,17632 1434340848
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,17632 1434340895
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,17632 1434340895


In [481]:
learning_time_threshold =  3*24*60*60
learning_time_threshold

259200

In [482]:
user_learning_time_threshold = user_min_time.user_id.map(str) + ' ' + (user_min_time.min_timestamp + learning_time_threshold).map(str)

In [483]:
user_learning_time_threshold.head( ) # серия, в которой для каждого пользователя проставленая его максимальное значение времени и айдишник пользователя 
# после которого , мы уже не будем использовать

0    1 1473086664
1    2 1514642564
2    3 1434617676
3    5 1466416009
4    7 1521893860
dtype: object

In [484]:
user_min_time['user_learning_time_threshold'] = user_learning_time_threshold

In [485]:
events_data = events_data.merge(user_min_time[['user_id', 'user_learning_time_threshold']], how = 'outer')

In [486]:
events_data.shape # прибавилась +1 колонка 'user_learning_time_threshold'

(3480703, 8)

In [487]:
events_data.head( )

,step_id,timestamp,action,user_id,date,day,user_time,user_learning_time_threshold
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,17632 1434340848,17632 1434600048
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,17632 1434340848,17632 1434600048
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,17632 1434340848,17632 1434600048
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,17632 1434340895,17632 1434600048
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,17632 1434340895,17632 1434600048


In [488]:
events_data_train = events_data[events_data.user_time <= events_data.user_learning_time_threshold]

In [489]:
events_data_train.head( ) # создали датафрейм, в котором отобрали временные промежутки для каждого пользователя
# удовлетворяющие условию, что не могли решить задачу дольше 3х дней (значит уходили с курсов)

,step_id,timestamp,action,user_id,date,day,user_time,user_learning_time_threshold
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,17632 1434340848,17632 1434600048
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,17632 1434340848,17632 1434600048
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,17632 1434340848,17632 1434600048
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,17632 1434340895,17632 1434600048
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,17632 1434340895,17632 1434600048


In [490]:
events_data_train.groupby('user_id').day.nunique( ).max( )

4

In [491]:
submissions_data = pd.read_csv( 'D:\\КУРСЫ DATA SCIENCE\\МОИ ЗАДАЧКИ\\submissions_data_train.csv')
submissions_data.head( )

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


In [492]:
submissions_data['date'] = pd.to_datetime(submissions_data.timestamp, unit = 's') 
submissions_data.head ( )

,step_id,timestamp,submission_status,user_id,date
0,31971,1434349275,correct,15853,2015-06-15 06:21:15
1,31972,1434348300,correct,15853,2015-06-15 06:05:00
2,31972,1478852149,wrong,15853,2016-11-11 08:15:49
3,31972,1478852164,correct,15853,2016-11-11 08:16:04
4,31976,1434348123,wrong,15853,2015-06-15 06:02:03


In [493]:
submissions_data['day'] = submissions_data.date.dt.date
submissions_data.head (10)

,step_id,timestamp,submission_status,user_id,date,day
0,31971,1434349275,correct,15853,2015-06-15 06:21:15,2015-06-15
1,31972,1434348300,correct,15853,2015-06-15 06:05:00,2015-06-15
2,31972,1478852149,wrong,15853,2016-11-11 08:15:49,2016-11-11
3,31972,1478852164,correct,15853,2016-11-11 08:16:04,2016-11-11
4,31976,1434348123,wrong,15853,2015-06-15 06:02:03,2015-06-15
5,31976,1434348188,correct,15853,2015-06-15 06:03:08,2015-06-15
6,31976,1478852055,correct,15853,2016-11-11 08:14:15,2016-11-11
7,31977,1434347371,correct,15853,2015-06-15 05:49:31,2015-06-15
8,31978,1434349981,correct,15853,2015-06-15 06:33:01,2015-06-15
9,31979,1434446091,correct,15853,2015-06-16 09:14:51,2015-06-16


In [494]:
submissions_data = submissions_data.merge(user_min_time[['user_id', 'user_learning_time_threshold']], how='outer')

In [495]:
submissions_data.shape

(518398, 7)

In [496]:
submissions_data['user_time'] = submissions_data.user_id.map(str) + ' ' + submissions_data.timestamp.map(str)

In [497]:
submissions_data.head( )

,step_id,timestamp,submission_status,user_id,date,day,user_learning_time_threshold,user_time
0,31971.0,1.434349e+09,correct,15853,2015-06-15 06:21:15,2015-06-15,15853 1434605177,15853 1434349275.0
1,31972.0,1.434348e+09,correct,15853,2015-06-15 06:05:00,2015-06-15,15853 1434605177,15853 1434348300.0
2,31972.0,1.478852e+09,wrong,15853,2016-11-11 08:15:49,2016-11-11,15853 1434605177,15853 1478852149.0
3,31972.0,1.478852e+09,correct,15853,2016-11-11 08:16:04,2016-11-11,15853 1434605177,15853 1478852164.0
4,31976.0,1.434348e+09,wrong,15853,2015-06-15 06:02:03,2015-06-15,15853 1434605177,15853 1434348123.0


In [498]:
submissions_data_train = submissions_data[submissions_data.user_time <= submissions_data.user_learning_time_threshold]

In [499]:
submissions_data_train.groupby('user_id').day.nunique().max()

4

In [500]:
X=submissions_data_train.groupby('user_id').day.nunique( ).to_frame().reset_index() \
     .rename(columns = {'day' : 'days'})

In [501]:
steps_tried = submissions_data_train.groupby('user_id').step_id.nunique( ).to_frame().reset_index() \
    .rename(columns = {'step_id' : 'steps_tried'})

In [502]:
steps_tried.head( )

,user_id,steps_tried
0,2,2
1,3,4
2,8,11
3,14,1
4,16,23


In [503]:
X = X.merge(steps_tried, on='user_id', how='outer')

In [504]:
X.head( )

,user_id,days,steps_tried
0,2,1,2
1,3,1,4
2,8,1,11
3,14,1,1
4,16,3,23


In [505]:
submissions_data_train.pivot_table(index = 'user_id', 
                                    columns = 'submission_status',
                                    values = 'step_id',
                                    aggfunc = 'count', 
                                    fill_value = 0).reset_index( ).head( )
#Кол-во попыток для каждого пользователя для этого датафрейма за 3 дня

submission_status,user_id,correct,wrong
0,2,2,0
1,3,4,4
2,8,9,21
3,14,0,1
4,16,23,27


In [506]:
X = X.merge(submissions_data_train.pivot_table(index = 'user_id', 
                                    columns = 'submission_status',
                                    values = 'step_id',
                                    aggfunc = 'count', 
                                    fill_value = 0).reset_index( ))
X.head( )

,user_id,days,steps_tried,correct,wrong
0,2,1,2,2,0
1,3,1,4,4,4
2,8,1,11,9,21
3,14,1,1,0,1
4,16,3,23,23,27


In [507]:
X['correct_ratio'] = X.correct/(X.correct+X.wrong)
X.head( )

,user_id,days,steps_tried,correct,wrong,correct_ratio
0,2,1,2,2,0,1.00
1,3,1,4,4,4,0.50
2,8,1,11,9,21,0.30
3,14,1,1,0,1,0.00
4,16,3,23,23,27,0.46


In [508]:
X = X.merge(events_data_train.pivot_table(index = 'user_id', 
                                    columns = 'action',
                                    values = 'step_id',
                                    aggfunc = 'count', 
                                    fill_value = 0).reset_index( ) [['user_id', 'viewed']], how = 'outer')

In [509]:
X = X.fillna(0)

In [510]:
X = X.merge(users_data[[ 'user_id', 'passed_corse', 'is_gone_user']], how = 'outer')

In [511]:
X.head( )

,user_id,days,steps_tried,correct,wrong,correct_ratio,viewed,passed_corse,is_gone_user
0,2,1.0,2.0,2.0,0.0,1.00,9,False,True
1,3,1.0,4.0,4.0,4.0,0.50,20,False,True
2,8,1.0,11.0,9.0,21.0,0.30,154,False,True
3,14,1.0,1.0,0.0,1.0,0.00,9,False,True
4,16,3.0,23.0,23.0,27.0,0.46,132,True,True


In [512]:
#Зададим условие, при котором уберем пользователей, которые начали курс, но еще его не закончили/ 
# Выводит всех пользователей,КРОМЕ тех, кот. не дошли до конца курса и не бросили еще
X = X[~((X.is_gone_user == False) & (X.passed_corse ==  False))] # ~ отрицание

In [513]:
# Либо, он курс не прошел еще и не бросил его

In [514]:
X.head( )

,user_id,days,steps_tried,correct,wrong,correct_ratio,viewed,passed_corse,is_gone_user
0,2,1.0,2.0,2.0,0.0,1.00,9,False,True
1,3,1.0,4.0,4.0,4.0,0.50,20,False,True
2,8,1.0,11.0,9.0,21.0,0.30,154,False,True
3,14,1.0,1.0,0.0,1.0,0.00,9,False,True
4,16,3.0,23.0,23.0,27.0,0.46,132,True,True


In [515]:
X.groupby(['passed_corse', 'is_gone_user']).user_id.count( )

passed_corse  is_gone_user
False         True            16560
True          False             141
              True             1284
Name: user_id, dtype: int64

Все пользователи, которые не прошли курс (passed_corse = False), они все помечены , как дропнутые (is_gone_user = True)


In [516]:
y = X.passed_corse.map(int)

In [517]:
X = X.drop(['passed_corse', 'is_gone_user'], axis = 1)

In [518]:
X = X.set_index(X.user_id) # Продублировали юзер айди в индекс, после чего можно дропнуть эту колонку
X = X.drop(['user_id'], axis = 1)

In [519]:
X.head( )

,days,steps_tried,correct,wrong,correct_ratio,viewed
user_id,,,,,,
2,1.0,2.0,2.0,0.0,1.00,9
3,1.0,4.0,4.0,4.0,0.50,20
8,1.0,11.0,9.0,21.0,0.30,154
14,1.0,1.0,0.0,1.0,0.00,9
16,3.0,23.0,23.0,27.0,0.46,132


In [520]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split

In [521]:
from sklearn.model_selection import cross_val_score

In [522]:
# Логистическая регрессия
clf = LogisticRegressionCV(cv=5)

In [523]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=41)

In [525]:
clf.fit(X_train, y_train)

LogisticRegressionCV(cv=5)

In [526]:
print(clf.score(X_test,y_test))

0.926605504587156


In [527]:
print(clf.coef_)

[[ 6.09286432e-01 -3.90076236e-02  9.76739077e-02  5.52380992e-03
   9.75720690e-01 -9.50870784e-04]]


In [528]:
print(clf.intercept_)

[-4.05644242]


In [529]:
print(cross_val_score(clf,X_train,y_train,cv=4).mean())

0.9218098415346122


In [530]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

dt = DecisionTreeClassifier(criterion='entropy')
parameters = {'max_depth': range(3,6), 'max_leaf_nodes':range(6,15), 'min_samples_leaf': range(1,4),'min_samples_split':range(2,5)}
grid_search_cv_clf = GridSearchCV(dt,parameters,cv=4)
grid_search_cv_clf.fit(X_train,y_train)

model = grid_search_cv_clf.best_estimator_
print(grid_search_cv_clf.best_params_, cross_val_score(model,X_train,y_train,cv=4).mean())

{'max_depth': 5, 'max_leaf_nodes': 11, 'min_samples_leaf': 2, 'min_samples_split': 3} 0.9188907422852377


In [531]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

knn = KNeighborsClassifier()
parameters = {'n_neighbors': range(15,25), 'leaf_size':range(1,7)}
grid_search_cv_clf = GridSearchCV(knn,parameters,cv=4,n_jobs=-1)
grid_search_cv_clf.fit(X_train,y_train)

model = grid_search_cv_clf.best_estimator_
print(grid_search_cv_clf.best_params_, cross_val_score(model,X_train,y_train,cv=4).mean())

{'leaf_size': 1, 'n_neighbors': 19} 0.9194467611898804


Вариант 2

In [533]:
from sklearn import metrics

def all_metrics(model, X_test, y_test):
    observations = y_test
    prediction = model.predict(X_test)

    true_positive = np.logical_and(prediction, observations)
    false_positive = np.logical_and(prediction, np.logical_not(observations))
    true_negative = np.logical_and(np.logical_not(prediction), np.logical_not(observations))
    false_negative = np.logical_and(np.logical_not(prediction), observations)

    tp, fp, tn, fn = (x.sum() for x in (true_positive, false_positive, true_negative, false_negative))
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2 * (recall * precision) / (recall + precision)

    print('True Positive: ', tp)
    print('False Positive: ', fp)
    print('True Negative: ', tn)
    print('False Negative: ', fn)
    print('Accuracy: ', accuracy)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1 score: ', f1_score)
    
    metrics.plot_roc_curve(model, X_test, y_test)
    metrics.plot_precision_recall_curve(model, X_test, y_test)
    metrics.plot_confusion_matrix(model, X_test, y_test)